<center><b><h1> Stacked Approximation Regression Machines from First Principles </b></h1> (Gabriel Goh)</center>

<img src = "bases.png">

There has been some buzz on [reddit](https://www.reddit.com/r/MachineLearning/comments/50tbjp/stacked_approximated_regression_machine_a_simple/) about the paper [Stacked Approximated Regression Machine: A Simple Deep Learning Approach](https://arxiv.org/abs/1608.04062). Approaching the paper with a measured dose of skepticism, I was pleasantly surprised to find the paper containing a beautiful kernel of an idea - one I can see becoming a fixture in our deep learning toolkit, the $k$-Approximated Regression Machine ($k$-ARM). 

### Background - Deep Nets

A deep neural network is a function which takes as input, $x$, data, into an output, $F(x)$. The word “deep” in deep learning refers to the network being a composition of layer functions:
$$F(x)=\Psi^{m}(\,\Psi^{m-1}(\,\cdots\,\Psi^{1}(x)\,\cdots\,)\,)$$

A traditional choice of a layer function looks like $\Psi^{k}(x)=\sigma(W^{T}_k x)$. Here $W^{T}_k$ is a matrix, representing a linear transform, (such as a convolution or a fully connected layer) and $\sigma$ is a choice of a non-linear [activation function](https://en.wikipedia.org/wiki/Activation_function). The goal in deep learning is to shape our function $F$, by any means possible, by tweaking the weights till they fit the data.

### The Regression Machine

While traditional tools worked well, there has been recently a Cambrian explosion of nontraditional layers. Consider the following conceptual layer. This layer isn't defined explicitly, but is instead defined implicitly as the solution of a *sparse coding problem*:
$$\Psi(x)=\underset{y}{\mbox{argmin }}\big\{\tfrac{1}{2}\|Wy-x\|^{2}+\lambda\|y\|_{1}\big\}.$$

I dub this the regression machine. 

This layer behaves very differently from a regular deep learning layer and it might be worthwhile to consider what it does. $W$ is a set of weights, to be trained, and $\lambda$ is a parameter controlling how sparse the output is. Assuming a trained model, with sensible $W$, this layer is a sparse encoder. It takes $x$, it's input and transforms it into its representation in structural primitives (the columns of $W$). 

It is useful to think of a musical metaphor. The sound of a chord (our input $x$) is generated by a sparse superposition of sounds corresponding to a small number of notes (our sparse parameter $y$). Our layer takes this generative process and throws it in reverse, and attempts to, from the sound of a chord, to recover the individual notes. 

If you have a background in engineering, these terms might be familiar to you in a different language. The generative process can be seen as the forward model, 

<img src="sparse.svg" width = 450px>

and the problem of recovering the notes from its output, the inverse problem

<img src="inverse.svg" width = 550px>

This layer is therefore, paradoxically, the opposite of a regular deep learning layer. But it is this lopsided layer which makes the most sense, as it truly takes data from input space into explanation space - a higher level of abstraction. It's no surprise then that this is the mechanism proposed by [Olshausen et al](http://redwood.psych.cornell.edu/papers/olshausen_field_1997.pdf) to represent the early stages of visual processing.

Despite its implicit definition, we can still take its (sub) gradient,
$$\begin{align}\nabla\Psi(x) & =W^{T}(Wy^\star-x)+\lambda\mbox{sign}(y^\star)\\
\frac{\partial}{\partial W_{ij}}\Psi(x) & =\frac{\partial}{\partial W_{ij}}\frac{1}{2}\|Wy^\star-x\|^{2}
\end{align}$$

by plugging $y^\star = \Phi(x)$, the solution to the sparse coding problem. Hence there is no technical roadblock in integrating this into a deep learning framework. 

This leads to the troubling question - why isn't this kind of layer de-facto? The trouble with this layer is that it is expensive. Computing this layer requires the solution of a small optimization problem (on the order of the number of features in a datum). Measured in milliseconds, this may be cheap -  the cost of solving this problem is on the order of solving a linear system. But it is still an order of magnitude more expensive than a simple matrix multiplication passed through a nonlinearity. Adding a single layer like this would potentially increase the cost of every training step a hundredfold.

### The $k$-ARM

Let us instead try to approximate this layer. First, I define a (seemingly) cryptic operator
$$
\Psi_{x}'(y) := \sigma(y-\alpha W^{T}(Wy-x)), \qquad \sigma(y)_{i}=\mbox{sign}(y_i)\max\{0,\left| y_{i} \right|-\lambda\}
$$

where $\alpha$ is a number strictly greater than zero, but smaller than the largest eigenvector of $W^{T}W$. Let's think about what this operator does. Let $f(y)=\tfrac{1}{2}\|Wy-x\|^{2}$, our quadratic term in the objective. The term inside the $\sigma$ is then just $y - \alpha\nabla f(y)$, a gradient step. The nonlinearity, $\sigma$, snaps all components of this new iterate between $[-\lambda, \lambda]$ to zero. So our operator consists of two steps. The first step improves the objective, and the second promotes sparsity. Furthermore, let's take these two facts as given:

1. $\Psi_x$ is the unique fixed point of the map $y\mapsto \Psi_{x}'(y),$ i.e. $\Psi(x)=\Psi_{x}'(\Psi(x))$
2. From any inital point $y$, repeated application of the map will always converge to $\Psi(x).$

In informal but suggestive notation, 
$$\Psi(x)=\Psi_{x}'(\Psi_{x}'(\Psi_{x}'(\cdots))).$$

Which suggests a series of approximations $\Psi\approx\Psi_{k}$, (starting at $0$ for simplicity),
$$\begin{align}
\Psi_{0}(x) & =\Psi_{x}'(\mathbf{0})\\
\Psi_{1}(x) & =\Psi_{x}'(\Psi_{x}'(\mathbf{0}))\\
\Psi_{2}(x) & =\Psi'_{x}(\Psi_{x}'(\Psi_{x}'(\mathbf{0})))\\
 & \vdots\\
\lim_{k\rightarrow\infty}\Psi_{k}(x) & =\Psi(x).
\end{align}.$$

This approximation has an architectural diagram looking like this

<p>
<img src = "diagram.svg" width = 500px>
<p>

And our most aggressive approximation, $\Psi_0(x)$ has the form
$$
\Psi_{0}(x)	
  =\Psi_{x}'(\mathbf{0})
  =\sigma(\mathbf{0}-\tfrac{1}{\alpha}W^{T}(W\mathbf{0}-x))
  =\sigma(\tfrac{1}{\alpha}W^{T}x)
$$

which should look familiar. Why, it's nothing more than our traditional layer discussed at the beginning! A tantalizing coincidence.

### The Generalized $k$-ARM

##### Proximal Gradient
Where does the operator $\Psi_{x}'(y)$ come from? There is, in fact, a powerful framework in convex optimization which gives us the tools to craft such operators. The study of proximal operators gives us a means to solve problems of the form
$$y^\star = \mbox{argmin}\{f(y)+g(y)\}$$

Where $f$ is smooth and convex and $g$ is just convex. For reasons which will be clear later, $f$ is usually chosen to be the vessel of our data, and $g$ to be structurally simple, like the 1-norm. Sparse coding, discussed earlier, is a special case of this where
$$f(y)=\tfrac{1}{2}\|Wy-x\|^{2},\qquad g(y)=\lambda\|y\|_{1}$$

This framework gives us a recipie on how to replace the 1-norm with any convex function $g$, such as any p-norm, grouped 2-norms and more.
##### The Proximal Gradient Iteration

Given $f$ and $g$, the proximal gradient method defines the map
$$\Psi_{f}^{'}[g](x)=\sigma[g](y_{k}+\alpha\nabla f(y)),\qquad\sigma[g](y)=\mbox{argmin}\{\tfrac{1}{2}\|\bar{y}-y\|^{2}+g(y)\}$$

so that, you guessed it

1. $y^\star$ is the unique fixed point of the map $y\mapsto \Psi_{f}'[g](y)$, i.e. $y^\star = \Psi_{f}'[g](y^\star)$
2. From any $y$, for small enough $\alpha>0$ repeated application of the map will always converge to $y^\star$

See Boyd's [slides](https://people.eecs.berkeley.edu/~elghaoui/Teaching/EE227A/lecture18.pdf) for more details. A few notes are in order. First, though you can in principle stick in any $g$ you want, you cannot escape from having to compute $\sigma_g$. This method hinges on when $\sigma_g$ having something resembling a closed form solution. Second, this framework encompasses constrained optimization with a bit of syntactic sugar:
$$\underset{y\in S}{\mbox{argmin }}f(y)=\underset{y}{\mbox{argmin}}\{f(y)+\delta(y \, | \, S)\},\qquad\delta(y)=\begin{cases}
0 & y\in S\\
\infty & y\notin S
\end{cases}.$$

So the constrained problem can be "simulated" by using $g(y) = \delta(y \, | \, S)$.

For simple $g$'s, we can create a collection of its $\sigma_g$'s. Here is a sampling
$$\begin{alignat*}{2}
 & g(y)=0 &  & \sigma[g](y)=y\\
 & g(y)=\lambda\|y\|_{1} &  & \sigma[g](y)_{i}=\mbox{sign}(x_{i})(\left|x_{i}\right|-\lambda)_{i}\\
 & g(y)=\lambda\|y\|_{2} &  & \sigma[g](y)=\max\{0,1-\lambda/\|y\|_{2}\}y\\
 & g(y)=\delta(y,\mathbf{R}_{+}) & \qquad & \sigma[g](y)_{i}=\max\{0,y_{i}\}\\
 & g(y)=\delta(y,\mathbf{R}_{+})+\lambda\|y\|_{1} & \qquad & \sigma[g](y)_{i}=\max\{0,y_{i}-\lambda\}
\end{alignat*}$$

##### A new cornucopia of ARMs

Now for each $g$, we have its corresponding Regression Machines, 
$$\Psi[g](x)=\underset{y}{\mbox{argmin }}\big\{\tfrac{1}{2}\|Wy-x\|^{2}+g(y)\big\}$$
With corrosponding ARMs defined using the iterations of the map
$$\Psi[g]_{x}'(y):=\sigma[g](y-\alpha W^{T}(Wy-x))$$

Take for example the Non-Negative Regression Machine, with $g(x)=\delta(x,{\mathbf R}_+)$. This has
$$\Psi[\delta(\cdot|\mathbf{R}_{+})]=\underset{y\geq0}{\mbox{argmin }}\tfrac{1}{2}\|Wy-x\|^{2},\quad\Psi[\delta(\cdot|\mathbf{R}_{+})]_{x}'(y):=\max\{0,y-\alpha W^{T}(Wy-x)\}$$
With a 0-ARM of
$$\Psi[\delta(\cdot|\mathbf{R}_{+})]_{0}(y)=\max\{0,W^{T}y\},$$

exactly a traditional layer with `ReLu` Activation.

### A note on training the Network

Finding the $W$'s for the network still remains a nagging problem. The usual set of tools apply, and given a large labeled dataset, the application of back-propagation on the $k$-ARM still works the way it should.

##### Initialization via Matrix Factorization

The paper proposes a heuristic for initializing $W$'s based on Dictionary Learning. For a single regression machine, assuming a dataset {$x_1,\dots,x_n$} which possesses sparse linear structure, one could try to learn both the $y_i$'s and $W$'s simultaneously in the joint optimization problem
$$\underset{W,y_{1},\dots,y_{n}}{\mbox{argmin}}\left\{ \sum_{i=1}^{n}\left(\tfrac{1}{2}\|Wy_{i}-x_{i}\|^{2}+\lambda\|y_{i}\|_{1}\right)\right\}.$$

This is known as the [*Dictionary Learning Problem*](https://en.wikipedia.org/wiki/Sparse_dictionary_learning). Similarly, the Non-Negative ARM can be initialized via [Non-Negative Matrix Factorization](https://en.wikipedia.org/wiki/Non-negative_matrix_factorization)
$$\underset{W\geq0,y_i\geq0}{\mbox{argmin}}\left\{ \sum_{i=1}^{n}\left(\tfrac{1}{2}\|Wy_{i}-x_{i}\|^{2}\right)\right\} $$

assuming the data is positive entrywise.

These methods are all modifications of the mother of unsupervised learning, Principle Component Analysis. Like PCA, you are trying to find an explanation of the data as a small linear combinations of vectors. Unlike PCA, we have extra restrictions, like sparsity or non-negativity. The paper suggests finding a local solution to Dictionary Learning problem using $k$-PCA.

Initialization for the a stacked ARM works as follows

- Take the raw data, $x_i$, and solve the Dictionary Learning problem to yield $W$ and $y_i$'s.
- Use $y_i$'s as the raw data for the next layer, and repeat.

This technique eerily recalls our forgotten hero the [Restricted Boltzman Machine (RBM)](https://en.wikipedia.org/wiki/Restricted_Boltzmann_machine). Like the RBM, it too is a form of unsupervised pre-training. And like the RBM, it too learns weights greedily. Unlike the RBM, we don't need to deal with the uncomfortable transplantation of the weights from a RBM to a deep, directed network. The $k$-ARM unifies both initialization and inference, making our weight surgery less jarring. 

This rationalization may soothe those of us who crave order in our souls, but the most valuable proof is in the pudding. The stacked ARMs show exciting results on training data, and is a great first step in what I see as an exciting direction of research.


Oh, and here's a link to my [website](http://gabgoh.github.io).

Picture in the title, as well as the sparse encoding picture, is taken from [Olshausen et al, Sparse Coding With An Overcomplete Basis Set: A Strategy Employed by V12?](http://redwood.psych.cornell.edu/papers/olshausen_field_1997.pdf)

In [2]:
from IPython.core.display import HTML
HTML(open("style.css","r").read())